In [1]:
### This project essentially captures a search feature for audio files.
### The following program allows you to search for a particular sound (series of sound)
### from the file you wanted to search for.

In [2]:
from scipy import fft, arange, signal
from scipy.special import logit, expit
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
import os
from ttictoc import TicToc

In [3]:
def read_file(path):
    """
    Reads the audio .wav file and returns the sample rate and data contents of the file.
    """
    sr, signal = wavfile.read(path)
    return sr, signal[:,0]

In [4]:
def read_all(files):
    """Branch audio file extrapolation. Uses read_file() method."""
    ret = []
    t = TicToc()
    t.tic();
    for fl in files:
        ret.append(read_file(fl))
    t.toc();
    print("Time it took to read data from files of length ", len(files), " = ", round(t.elapsed, 3), " seconds.")
    return np.array(ret)

In [5]:
def frequency_sepectrum(sf, x):
    """
    Derive frequency spectrum of a signal from time domain
    :param x: signal in the time domain
    :param sf: sampling frequency
    :returns frequencies and their content distribution
    """
    x = x - np.average(x)  # zero-centering

    n = len(x)
    k = arange(n)
    tarr = n / float(sf)
    frqarr = k / float(tarr)  # two sides frequency range

    frqarr = frqarr[range(n // 2)]  # one side frequency range

    x = fft(x) / n  # fft computing and normalization
    x = x[range(n // 2)]

    return frqarr, abs(x)

In [6]:
def plot(sample_rate, signal, state=False):
    """
    Method responsible for converting from time domain to frequency
    via the discrete fourier analysis function created in frequency_sepectrum().
    Option to form a plot of the resultant frequency domain graph with its frequency content distribution.
    Returns:
    1. Y: The frequencies.
    2. frq: The content distribution.
    """
    frq, Y = frequency_sepectrum(sample_rate, signal)
    frq = frq
    Y = Y
    if(state == True):
        plt.plot(frq, Y)
    return Y, frq

In [7]:
def fourier_transformation(sample_rates, signals):
    """
    Return Foureir Transformation for each individual plot.
    Note: the first index is reserved for Test file.
    Parameters: 
    1. sample_rates: np.array() array 
    """
    Y = []
    frequencies = []
    for sr,sig in zip(sample_rates, signals):
        content_distribution, frq = plot(sr, sig)
        Y.append(frq)
        frequencies.append(content_distribution)
    return np.array(Y), np.array(frequencies)

In [55]:
def sort_list(lists):
    """
    Sorts a dictionary based on values().
    Return the sorted dictionary. Accepts an unsorted dictionary.
    """
    listofTuples = sorted(lists.items() , reverse=False, key=lambda x: x[1])
    # Iterate over the sorted sequence
    data = []
    for elem in listofTuples :
        data.append((elem[0], elem[1]))  
    return data

In [56]:
def find_deviation(point, tests, titles):
    """
    Finds the deviation from the np.array() 'point' based on individual np.array() 'tests'.
    """
    delta = {}
    t = TicToc()
    t.tic()
    for test, title in zip(tests, titles):
        _, one_res = percent_calculator(point, test)
        delta[title] = one_res
    t.toc()
    print("Time taken(s): ", round(t.elapsed, 3))
    return sort_list(delta), delta

In [43]:
def percent_calculator(A, B):
    """
    Finds the distribution from SAMPLE based on TEST data.
    """
    percent = []
    A = A #+ np.linalg.norm(A)
    B = B #+ np.linalg.norm(B)
    for a,b in zip(A, B):
        percent.append(abs(a-b))
    tots = 1
    if(sum(percent) > 1):   
        tots = np.log(sum(percent)) #sum(percent)/(np.linalg.norm(A))
    return np.array(percent), tots

### main() calibration

In [44]:
def retrieve_all_files(path=None):
    """
    Returns a list of audio files with the extension, .wav, as an np.array() object.
    Parameters:
    1. path (current directory if left empty): The path of directory to scrape through. str object.
    """
    import os
    files = os.listdir(path)
    refurbished_list = []
    for file in files:
        if(file.endswith(".wav")):
            refurbished_list.append(file)
    return np.array(refurbished_list)

In [45]:
files = retrieve_all_files()
data = read_all(files)

Time it took to read data from files of length  9  =  0.273  seconds.


In [46]:
def retrieve_sample_signals(data):
    """
    Splits data(above) into signal and sample_rate arrays.
    """
    dsp = []
    sr = []
    for df in data:
        sr.append(df[0])
        dsp.append(df[1])
    return sr, dsp

In [57]:
SR, DSP = retrieve_sample_signals(data)

In [58]:
frequencies, content_distribution = fourier_transformation(SR, DSP)

In [64]:
result, unsorted_result = find_deviation(content_distribution[-1], content_distribution, files)

Time taken(s):  43.394


In [65]:
unsorted_result

{'Animals Live Test.wav': 15.03670298079866,
 'Animals,  Shazam Project Test.wav': 15.266952438366665,
 'Animals, batch test 1.wav': 14.370613909369341,
 'Dynoro & Gigi DAgostino - In My Mind.wav': 15.060930791498166,
 'Ed Sheeran - Thinking Out Loud (x Acoustic Session).wav': 14.813706616810281,
 'Ed Sheeran - Thinking Out Loud - (Live At Capitals Jingle Bell Ball 2017).wav': 14.875111708640592,
 'Martin Garrix - Animals (test#1).wav': 15.267213821598917,
 'Marvin Gaye - Lets get it on.wav': 14.955179485899336,
 'Thinking out Loud.wav': 1}

In [66]:
for f in files:
    print(f)

Animals Live Test.wav
Animals,  Shazam Project Test.wav
Animals, batch test 1.wav
Dynoro & Gigi DAgostino - In My Mind.wav
Ed Sheeran - Thinking Out Loud (x Acoustic Session).wav
Ed Sheeran - Thinking Out Loud - (Live At Capitals Jingle Bell Ball 2017).wav
Martin Garrix - Animals (test#1).wav
Marvin Gaye - Lets get it on.wav
Thinking out Loud.wav


In [67]:
files[1]

'Animals,  Shazam Project Test.wav'

In [68]:
result

[('Thinking out Loud.wav', 1),
 ('Animals, batch test 1.wav', 14.370613909369341),
 ('Ed Sheeran - Thinking Out Loud (x Acoustic Session).wav',
  14.813706616810281),
 ('Ed Sheeran - Thinking Out Loud - (Live At Capitals Jingle Bell Ball 2017).wav',
  14.875111708640592),
 ('Marvin Gaye - Lets get it on.wav', 14.955179485899336),
 ('Animals Live Test.wav', 15.03670298079866),
 ('Dynoro & Gigi DAgostino - In My Mind.wav', 15.060930791498166),
 ('Animals,  Shazam Project Test.wav', 15.266952438366665),
 ('Martin Garrix - Animals (test#1).wav', 15.267213821598917)]

In [69]:
#Reflection... Kinda works, need to work on implementing a better percent modulization curve model.